# Simulación
Dada la complejidad, los trabajadores de la empresesa decidieron desarrollar parte de su código en la plataforma "Notebook de python"

In [9]:
type Bolita = 
    {
        x: float
        y: float
        vx: float
        vy: float
    }

type Barra = 
    {
        x: float
        y: float
        Lx: float
    }

let xP1 = 0.
let yP1 = 0.
let xP2 = 100.
let yP2 = 100.
let dt = 1e-3

type Bloque = 
    {
        x: float
        y: float
        Lx: float
        Ly: float
    }

In [5]:
let condicion_inicial_bolita = 
    let bolita:Bolita = {
        x = 1.
        y = 1.
        vx = 0.
        vy = 1.
    }
    bolita

let condicion_inicial_bloques =
    let bloque:Bloque = {
        x = 0.
        y = 0.
        Lx = 10.
        Ly = 10.
    }
    [bloque;bloque] 


let actualiza_bolita (bolita:Bolita) =
    bolita

In [8]:
let bolita:Bolita = condicion_inicial_bolita

let bloques:Bloque List = condicion_inicial_bloques

//Loop
let new_bolita = actualiza_bolita bolita

//Interacciones

